In [1]:
import numpy as np
import tensorflow as tf

In [2]:
x = tf.placeholder("float", shape=[None,32,32,3])
y_ = tf.placeholder(tf.int32, shape=[None])
keep_prob = tf.placeholder("float")
training_phase = tf.placeholder(tf.bool)

In [3]:
def im_pre(image):
    new_img = tf.image.random_brightness(image, max_delta=63) #随机调节图像的亮度  
    new_img = tf.image.random_flip_left_right(new_img) #随机地左右翻转图像  
    new_img = tf.image.random_contrast(new_img, lower=0.2, upper=1.8) #随机地调整图像对比度  
    new_img = tf.image.per_image_standardization(image)
    return new_img

In [4]:
img =  tf.map_fn(im_pre,x)

In [5]:
b_size = 128
learning_rate = 0.0001

In [6]:
reg = tf.contrib.layers.l2_regularizer(scale=0.1) 

In [7]:
conv_2d_w1 = tf.Variable(tf.truncated_normal([5, 5, 3, 32],stddev=0.0001))
conv_2d_b1 =  tf.Variable(tf.truncated_normal([32]))
conv2d_1 = tf.nn.conv2d(img, conv_2d_w1,strides=[1, 1, 1, 1], padding='SAME') + conv_2d_b1

In [8]:
conv2d_1 = tf.contrib.layers.batch_norm(conv2d_1,decay=0.96, is_training=training_phase)

In [9]:
conv2d_1_output =  tf.nn.relu(conv2d_1)

In [10]:
pool_1 = tf.nn.max_pool(conv2d_1_output,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [11]:
conv_2d_w2 = tf.Variable(tf.truncated_normal([5, 5, 32, 32],stddev=0.001))
conv_2d_b2 =  tf.Variable(tf.truncated_normal([32]))
conv2d_2 = tf.nn.conv2d(pool_1, conv_2d_w2,strides=[1, 1, 1, 1], padding='SAME') + conv_2d_b2

In [12]:
conv2d_2 = tf.contrib.layers.batch_norm(conv2d_2,decay=0.96, is_training=training_phase)

In [13]:
conv2d_2_output =  tf.nn.relu(conv2d_2)

In [14]:
pool_2 = tf.nn.max_pool(conv2d_2_output,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [15]:
conv_2d_w3 = tf.Variable(tf.truncated_normal([5, 5, 32, 64],stddev=0.01))
conv_2d_b3 =  tf.Variable(tf.truncated_normal([64]))
conv2d_3 = tf.nn.conv2d(pool_2, conv_2d_w3,strides=[1, 1, 1, 1], padding='SAME') + conv_2d_b3

In [16]:
conv2d_3 = tf.contrib.layers.batch_norm(conv2d_3,decay=0.96, is_training=training_phase)

In [17]:
conv2d_3_output =  tf.nn.relu(conv2d_3)

In [18]:
pool_3 = tf.nn.max_pool(conv2d_3_output,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [19]:
pool3_flat = tf.reshape(pool_3, [-1, 4*4*64])

In [20]:
fc_w1 = tf.Variable(tf.truncated_normal([4*4*64,1024],stddev=0.1))
fc_b1 =  tf.Variable(tf.truncated_normal([1024]))
fc_1 = tf.matmul(pool3_flat,fc_w1) + fc_b1

In [21]:
fc_1 = tf.contrib.layers.batch_norm(fc_1,decay=0.96, is_training=training_phase)

In [22]:
fc_1_output =  tf.nn.relu(fc_1)

In [23]:
fc_w2 = tf.Variable(tf.truncated_normal([1024,128],stddev=0.1))
fc_b2 =  tf.Variable(tf.truncated_normal([128]))
fc_2 = tf.matmul(fc_1_output,fc_w2) + fc_b2

In [24]:
fc_2 = tf.contrib.layers.batch_norm(fc_2,decay=0.96, is_training=training_phase)

In [25]:
fc_2_output =  tf.nn.relu(fc_2)

In [26]:
fc2_drop = tf.nn.dropout(fc_2_output, keep_prob)

In [27]:
out_w1 = tf.Variable(tf.truncated_normal([128,10]))
out_b1 = tf.Variable(tf.truncated_normal([10]))
combine = tf.matmul(fc2_drop,out_w1)+out_b1
pred = tf.cast(tf.argmax(tf.nn.softmax(combine),1),tf.int32)

In [28]:
weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [29]:
reg_ws = tf.contrib.layers.apply_regularization(reg,weights_list = weights)

In [30]:
#reg_ws = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

In [31]:
update_avg = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [32]:
loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=combine))

In [33]:
#loss = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(pred,1e-11,1.0)))
loss_fn = loss + tf.reduce_sum(reg_ws)

In [34]:
with tf.control_dependencies(update_avg):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss_fn)

In [35]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred,y_), tf.float32))

In [36]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [39]:
data_list = []
label_list = []
for i in range(1,6):
    data = unpickle('./cifar-10/data_batch_{}'.format(i))
    data_list.append(data[b'data'])
    label_list.append(data[b'labels'])

In [40]:
all_data = np.concatenate(data_list)
all_label = np.concatenate(label_list)

In [37]:
def generatebatch(X,Y,n_examples, batch_size):
    for batch_i in range(n_examples // batch_size):
        start = batch_i*batch_size
        end = start + batch_size
        batch_xs = X[start:end]
        batch_ys = Y[start:end]
        yield batch_xs, batch_ys # 生成每一个batch

In [42]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
ii = 0
for epoch in range(0,150): 
    index = np.random.permutation(all_label.shape[0])
    all_data = all_data[index]
    all_label = all_label[index]
    for batch_xs,batch_ys in generatebatch(all_data,all_label,all_label.shape[0],b_size): 
        batch_xs = np.array(list(map(lambda x:x.reshape([3,1024]).T.reshape([32,32,3]),batch_xs)))
        sess.run(train_step,feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 0.5, training_phase: True})
        if ii%100 == 0:
            print(sess.run([loss,accuracy,],feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 1, training_phase: True}))
        ii += 1
    if(epoch%2==0):
        res = sess.run([loss,accuracy],feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 1, training_phase: True})
        print(epoch,res)
        saver.save(sess,'./lesson41',global_step = epoch)

[1369.6284, 0.140625]
[628.1438, 0.3125]
[547.77747, 0.3984375]
[355.21271, 0.4453125]
0 [490.77081, 0.375]
[386.22327, 0.390625]
[345.26874, 0.4609375]
[253.89771, 0.484375]
[250.1356, 0.4765625]
[314.26764, 0.4765625]
[290.87585, 0.4921875]
[222.46878, 0.5078125]
[329.89886, 0.484375]
2 [237.0118, 0.515625]
[248.05194, 0.578125]
[178.05533, 0.578125]
[273.87378, 0.5234375]
[206.06012, 0.5625]
[192.15402, 0.5859375]
[202.27774, 0.59375]
[307.88812, 0.4921875]
[156.87416, 0.5859375]
4 [152.72061, 0.6484375]
[166.65565, 0.5546875]
[139.55022, 0.625]
[141.12291, 0.640625]
[199.5015, 0.6015625]
[179.99884, 0.6328125]
[176.64891, 0.5234375]
[133.9993, 0.6484375]
[186.08781, 0.6171875]
6 [130.98169, 0.7109375]
[159.3278, 0.6171875]
[126.03557, 0.703125]
[162.40244, 0.6875]
[198.90579, 0.6484375]
[115.09143, 0.703125]
[206.66866, 0.6015625]
[150.11266, 0.6875]
[170.43134, 0.6171875]
8 [156.30328, 0.6328125]
[121.53514, 0.6953125]
[134.25955, 0.640625]
[174.09216, 0.6640625]
[159.58595, 0.640

KeyboardInterrupt: 